# Diffusion gradients in Bruker data sets
## or: How I learned to stop worying and love coordinate systems

**WARNING** This is a work in progress. The informations should be completed with
* Check that the RPF to RPH conversion is valid in other data sets
* Data sets from Paravision 5
* Data from the noodles phantom

The official documentation from Bruker (Operating Manual of Paravision 6.0.1 and *ParaVision Parameters*) are lacking details to understand which coordinate system (gradient, subject, magnet or image) is used when manipulating vectors and matrices. This is especially difficult in DWI acquisitions, where the direction of the diffusion gradient is of the utmost importance in DEC (directionally-encoded color) images and tractography. The documentation is even self-contradicting for some fields: `ACQ_grad_matrix` is described as the transformation from gradient coordinates to subject coordinates in the Operating Manual (section 3.2.6.10.1) and from the gradient coordinates to patient-indenpendent coordinates in *ParaVision Parameters* (page D-2-36).

Based two diffusion data sets provided by Mathieu Santin and Chrystelle Po, we try to understand how to convert between different coordinate systems and how to express the directions of the diffusion gradient in these systems. In addition to core Python modules, [Dicomifier](https://github.com/lamyj/dicomifier/) and [NumPy](http://www.numpy.org/) are required.

The two data sets are respectively:
* A 3D acquisition with readout gradient on the head-tail axis, phase gradient on the left-right axis, and slice gradient on the ventro-dorsal axis.
* Seven 2D acquisitions with different orientations:
  * Coronal slices: readout on left-right and ventro-dorsal axes
  * Sagittal slices: readout on rostro-caudal and ventro-dorsal axes
  * Horizontal slices: readout on rostro-caudal and left-right axes
  * Arbitrarilly-oriented slices

The following anatomical orientations are used in this document:
* *L*, *R*: left and right of the subject
* *D*, *V*: dorsal (anterior or back) and ventral (posterior or front) sides of the subject
* *F*, *H*: foot (inferior or caudal) and head (superior or rostral) extremities of the subject

We start by defining the orientations as above, and loading the main meta-data files (`acqp`, `method`, and `visu_pars`) of each data set:

In [1]:
import glob
import itertools
import os
import subprocess
import sys

import dicomifier
import numpy

# Orientations of the gradients (readout, phase, slice) relative to the subject
gradient_orientations = {
    "saimiri": {
        7: ["F-H", "L-R", "V-D"],
    },
    "rat": {
        # Coronal slices
        6: ["L-R", "V-D", "F-H"],
        8: ["V-D", "L-R", "F-H"],
        # Sagittal slices
        9: ["F-H", "V-D", "L-R"],
        10: ["V-D", "F-H", "L-R"],
        # Horizontal slices
        11: ["F-H", "L-R", "V-D"],
        12: ["L-R", "F-H", "V-D"],
        # Arbitrary
        13: ["arbitrary", "arbitrary", "arbitrary"],
    }
}

meta_data = {}
meta_data_files = ["acqp", "method", "visu_pars"]
for name, series in gradient_orientations.items():
    for series_number in series:
        for file_ in meta_data_files:
            data_set = dicomifier.bruker.Dataset()
            data_set.load(
                os.path.join(name, "bruker", str(series_number), file_))
            
            meta_data.setdefault(name, {}).setdefault(series_number, {})
            meta_data[name][series_number][file_] = data_set

We then define a helper function to fetch n-dimensional numeric arrays from Bruker data sets.

In [2]:
def get_array(field, shape=None):
    return numpy.reshape(
        [field.get_float(x) for x in range(numpy.cumprod(field.shape)[-1])], 
        field.shape)

## Coordinate systems

According to the Operating Manual (section 3.2.6.10.1), the gradient coordinate system has a (*readout*, *phase*, *slice*) basis, and the transformation matrix from this coordinate system to the subject coordinate system (i.e. the coordinates of the gradient basis in the subject basis) is stored in `ACQ_grad_matrix`. The equation in the aforementioned section uses a $v\cdot M$ formalism, which points to the use of row-vectors, and ParaVision Parameters (pages D-2-36 and D-2-65) mentions matrices stored in the row-major order. The subject coordinate system is described by Bruker (Operating Manual, sections 1.3.6.1, 1.3.6.2, and 3.2.6.10.1) to be (*L* &rarr; *R*, *V* &rarr; *D*, *F* -> *H*); note that this is a [left-handed](https://en.wikipedia.org/wiki/Right-hand_rule) coordinate system. We compare the expected orientations with the ones stored in `ACQ_grad_matrix` using the dot product of the normalized vectors.

In [3]:
RDH = {
    "L-R": [1,0,0],
    "V-D": [0,1,0],
    "F-H": [0,0,1],
    "arbitrary": [0,0,0],
}

ACQ_grad_matrix = {}
for name, series in meta_data.items():
    for series_number, series_meta_data in series.items():
        ACQ_grad_matrix.setdefault(name, {})[series_number] = get_array(
            series_meta_data["acqp"].get_field("ACQ_grad_matrix"))

def compare_orientations(o1, o2):
    return numpy.dot(o1/numpy.linalg.norm(o1), o2/numpy.linalg.norm(o2))

for name, series in meta_data.items():
    for series_number, series_meta_data in series.items():
        if "arbitrary" in gradient_orientations[name][series_number]:
            continue
        
        dot_products = [
            compare_orientations(
                ACQ_grad_matrix[name][series_number][0][i], 
                RDH[gradient_orientations[name][series_number][i]])
            for i in range(3)
        ]
        print("{}/{}: {}".format(name, series_number, dot_products))

rat/6: [0.99930667480477031, 0.99930667480477031, 1.0]
rat/8: [0.99930667480477031, 0.99930667480477031, 1.0]
rat/9: [1.0, 1.0, 1.0]
rat/10: [1.0, 1.0, 1.0]
rat/11: [1.0, 1.0, 1.0]
rat/12: [1.0, 1.0, 1.0]
saimiri/7: [1.0, 1.0, 1.0]


With the exception of a small deviation in the coronal slices of the rat data set -- possibly due to a slice plane not perfectly aligned with the magnet axes -- all acquisitions match their definition. Note that for a same slice plane, switching the readout and phase gradient directions will yield left-hand frames or right hand coordinate systems.

In [4]:
data_set, series = "rat", 6
print("{}/{}: {}".format(
    data_set, series, numpy.linalg.det(ACQ_grad_matrix[data_set][series][0])))
data_set, series = "rat", 8
print("{}/{}: {}".format(
    data_set, series, numpy.linalg.det(ACQ_grad_matrix[data_set][series][0])))

rat/6: 1.00000000794
rat/8: -1.00000000794


The position matrices, converting from the subject RDH coordinate system to the magnet (or world) coordinate system, are described in ParaVision Parameters (page D-2-33).

In [5]:
position_matrix = {
    "Head_Supine": numpy.asarray([[-1, 0, 0], [0,  1, 0], [0, 0, -1]]),
    "Head_Prone":  numpy.asarray([[ 1, 0, 0], [0, -1, 0], [0, 0, -1]]),
    # ...
    "Feet_Supine": numpy.asarray([[ 1, 0, 0], [0,  1, 0], [0, 0,  1]]),
    "Feet_Prone":  numpy.asarray([[-1, 0, 0], [0, -1, 0], [0, 0,  1]]),
    # ...
}
ACQ_patient_pos = {}
for name, series in meta_data.items():
    for series_number, series_meta_data in series.items():
        matrix = position_matrix[
            series_meta_data["acqp"].get_field("ACQ_patient_pos").get_string(0)]
        ACQ_patient_pos.setdefault(name, {})[series_number] = matrix

Since `Feet_Supine` corresponds to an identity position matrix in the RDH subject coordinates, magnet axes are (when facing the tunnel):
* Left &rarr; Right
* Up &rarr; Down
* Back &rarr; Front

The last coordinate system is the image one where the axes are those of the voxel block. The transformation matrix from the subject coordinate system to the image coordinate system is stored in `VisuCoreOrientation`. It transforms from the *LDH* system in Paravision 6 and from the *RVH* system in Paravision 5 (Paravision Parameters, pages D-2-64 and D-2-65), using a *column vector* formalism.

In [6]:
is_pv6 = {
    name: series.values()[0]["visu_pars"].get_field("VisuCreatorVersion").get_string(0) > "6"
    for name, series in meta_data.items()
}

VisuCoreOrientation_coordinates = {
    name: (
        ["R->L", "V->D", "F->H"] if is_pv6[name] else ["L->R", "D->V", "F->H"])
    for name in meta_data
}

VisuCoreOrientation = {}
for name, series in meta_data.items():
    for series_number, series_meta_data in series.items():
        matrix = get_array(
                series_meta_data["visu_pars"].get_field("VisuCoreOrientation")
            ).reshape(-1, 3, 3)
        VisuCoreOrientation.setdefault(name, {})[series_number] = matrix

for name, series in meta_data.items():
    for series_number in series:
        print(
            "{}/{}: in image coordinates,\n"
            "   {}: {}\n"
            "   {}: {}\n"
            "   {}: {}".format(
                name, series_number, 
                *itertools.chain(*zip(
                    VisuCoreOrientation_coordinates[name],
                    VisuCoreOrientation[name][series_number][0].T))))

rat/6: in image coordinates,
   R->L: [ 0.99930668  0.0372313   0.        ]
   V->D: [-0.0372313   0.99930668  0.        ]
   F->H: [ 0.  0.  1.]
rat/8: in image coordinates,
   R->L: [ 0.99930668  0.0372313   0.        ]
   V->D: [-0.0372313   0.99930668  0.        ]
   F->H: [ 0.  0.  1.]
rat/9: in image coordinates,
   R->L: [ 0.  0. -1.]
   V->D: [ 1.  0.  0.]
   F->H: [ 0. -1.  0.]
rat/10: in image coordinates,
   R->L: [ 0.  0. -1.]
   V->D: [ 1.  0.  0.]
   F->H: [ 0. -1.  0.]
rat/11: in image coordinates,
   R->L: [ 1.  0.  0.]
   V->D: [ 0.  0.  1.]
   F->H: [ 0. -1.  0.]
rat/12: in image coordinates,
   R->L: [ 1.  0.  0.]
   V->D: [ 0.  0.  1.]
   F->H: [ 0. -1.  0.]
rat/13: in image coordinates,
   R->L: [ 0.99358684  0.10770127  0.03443334]
   V->D: [-0.1052677   0.99224889 -0.06603676]
   F->H: [-0.04127869  0.06198854  0.9972229 ]
saimiri/7: in image coordinates,
   R->L: [ 1.  0.  0.]
   V->D: [ 0.  0.  1.]
   F->H: [ 0. -1.  0.]


These axes can be checked by displaying the converted NIfTI image in FSLeyes or ITK-SNAP (the saimiri was positioned as head-first prone but but registered in Paravision as head-first supine, some axes might be flipped). On the rat data set, note that the contents of `VisuCoreOrientation` are the same for a given slice: the *x* and *y* image axes do not always correspond to respectively the readout and phase gradients. This behavior is governed by `RECO_transposition`.

## Diffusion gradient

The directions of the diffusion gradient are stored in all four coordinates systems (gradient, subject, magnet and image) in the various files:
* `PVM_DwDir`, `PVM_DwGradVec` and `PVM_DwBMat` use the gradient coordinate system (unless `PVM_DwDirectScale` is set to `Yes`, cf. Operating Manual, section 1.9.1.2)
* `PVM_DwBMatPat` and `VisuAcqDiffusionBMatrix` use the subject coordinate system
* `PVM_DwBMatMag` uses the magnet coordinate system
* `PVM_DwBMatImag` uses the image coordinate system

With the exception of `PVM_DwDir` and `PVM_DwGradVec`, which store the directions as 3D vectors, all other fields store [b-matrices ](http://doi.org/10.1006/jmrb.1994.1037). In this formalism, the diffusion gradient is colinear to the eigenvector associated with the eigenvalue having the largest absolute value (**warning** need to find a reference for this). The b-values of all volumes can be found in `PVM_DwEffBval`.

In [7]:
PVM_DwEffBval = {}
for name, series in meta_data.items():
    for series_number, series_meta_data in series.items():
        PVM_DwEffBval.setdefault(name, {})[series_number] = get_array(
            series_meta_data["method"].get_field("PVM_DwEffBval"))

### Gradient coordinate system

The contents of `PVM_DwDir` will serve as the ground truth, and will be transformed in each other coordinate system. Due to the use of eigenanalysis on the b-matrices, we will only obtain eigenvectors colinear to the diffusion gradient: the similarity check thus needs to use the *absolute value* of the dot product.

In [8]:
def check_colinearity(o1, o2):
    return numpy.abs(compare_orientations(o1, o2))

We also need to skip the b-matrices corresponding to acquisitions without a diffusion gradient, as those volumes are not present in `PVM_DwDir`.

In [9]:
PVM_DwDir = {}
PVM_DwGradVec = {}
PVM_DwBMat = {}
for name, series in meta_data.items():
    for series_number, series_meta_data in series.items():
        PVM_DwDir.setdefault(name, {})[series_number] = get_array(
            series_meta_data["method"].get_field("PVM_DwDir"))
        PVM_DwGradVec.setdefault(name, {})[series_number] = get_array(
            series_meta_data["method"].get_field("PVM_DwGradVec"))
        PVM_DwBMat.setdefault(name, {})[series_number] = get_array(
            series_meta_data["method"].get_field("PVM_DwBMat"))

b_value_threshold = 100

def get_orientation_and_b_value(b_matrix):
    eigenvalues, eigenvectors = numpy.linalg.eigh(b_matrix)
    eigenvalues = numpy.abs(eigenvalues)
    return eigenvectors[:,numpy.argmax(eigenvalues)], eigenvalues.max()

def get_diffusion_weighted_data(b_matrices, b_value_threshold):
    result = []
    for b_matrix in b_matrices:
        orientation, b_value = get_orientation_and_b_value(b_matrix)
        if b_value > b_value_threshold:
            result.append((orientation, b_value))
    return result

def report_colinearity(PVM_DwDir, b_matrices, transformer, b_value_threshold):
    for name, series in PVM_DwDir.items():
        for series_number, dw_dirs in series.items():
            b_matrix_data = get_diffusion_weighted_data(
                b_matrices[name][series_number], b_value_threshold)
            if len(b_matrix_data) != len(dw_dirs):
                raise Exception("Size mismatch")
            transform = transformer(name, series_number)
            colinearity = ([
                check_colinearity(numpy.dot(dw_dir, transform), orientation)
                for dw_dir, (orientation, _) in zip(dw_dirs, b_matrix_data)])
            min_colinearity = min(colinearity)
            max_angle = numpy.arccos(min_colinearity)
            print(
                "  {}/{}: {} ({} degrees)".format(
                    name, series_number, 
                    min_colinearity, numpy.degrees(max_angle)))

print("Minimal colinearity between PVM_DwDir and PVM_DwGradVec")
for name, series in PVM_DwDir.items():
    for series_number, dw_dirs in series.items():
        dw_grad_vecs = [
            x
            for x in PVM_DwGradVec[name][series_number]
            if numpy.linalg.norm(x)!=0]
        colinearity = ([
            check_colinearity(dw_dir, dw_grad_vec)
            for dw_dir, dw_grad_vec in zip(dw_dirs, dw_grad_vecs)])
        min_colinearity = min(colinearity)
        max_angle = numpy.arccos(min_colinearity)
        print(
            "  {}/{}: {} ({} degrees)".format(
                name, series_number, 
                min_colinearity, numpy.degrees(max_angle)))
print("Minimal colinearity between PVM_DwDir and PVM_DwBMat")
report_colinearity(
    PVM_DwDir, PVM_DwBMat, lambda x,y: numpy.identity(3), b_value_threshold)

Minimal colinearity between PVM_DwDir and PVM_DwGradVec
  rat/6: 1.0 (3.52019892386e-06 degrees)
  rat/8: 1.0 (3.52019892386e-06 degrees)
  rat/9: 1.0 (3.52019892386e-06 degrees)
  rat/10: 1.0 (3.52019892386e-06 degrees)
  rat/11: 1.0 (3.52019892386e-06 degrees)
  rat/12: 1.0 (3.52019892386e-06 degrees)
  rat/13: 1.0 (3.52019892386e-06 degrees)
  saimiri/7: 1.0 (5.05099300793e-06 degrees)
Minimal colinearity between PVM_DwDir and PVM_DwBMat
  rat/6: 0.995793159616 (5.25736362048 degrees)
  rat/8: 0.995793159616 (5.25736362048 degrees)
  rat/9: 0.995793159616 (5.25736362048 degrees)
  rat/10: 0.995793159616 (5.25736362048 degrees)
  rat/11: 0.995793159616 (5.25736362048 degrees)
  rat/12: 0.995793159616 (5.25736362048 degrees)
  rat/13: 0.995793159616 (5.25736362048 degrees)
  saimiri/7: 0.998381072353 (3.26069335517 degrees)


### Subject coordinate system

The contents of `PVM_DwBMatPat` and `VisuAcqDiffusionBMatrix` are the same.

In [10]:
PVM_DwBMatPat = {}
VisuAcqDiffusionBMatrix = {}
for name, series in meta_data.items():
    for series_number, series_meta_data in series.items():
        PVM_DwBMatPat.setdefault(name, {})[series_number] = get_array(
            series_meta_data["method"].get_field("PVM_DwBMatPat"))
        VisuAcqDiffusionBMatrix.setdefault(name, {})[series_number] = get_array(
                series_meta_data["visu_pars"].get_field("VisuAcqDiffusionBMatrix")
            ).reshape(-1, 3, 3)
print("Maximum difference between PVM_DwBMatPat and VisuAcqDiffusionBMatrix:")
for name, series in meta_data.items():
    for series_number in series:
        difference = numpy.abs(
            PVM_DwBMatPat[name][series_number]
            -VisuAcqDiffusionBMatrix[name][series_number])
        print("  {}/{}: {}".format(name, series_number, difference.max()))

Maximum difference between PVM_DwBMatPat and VisuAcqDiffusionBMatrix:
  rat/6: 0.0
  rat/8: 0.0
  rat/9: 0.0
  rat/10: 0.0
  rat/11: 0.0
  rat/12: 0.0
  rat/13: 0.0
  saimiri/7: 0.0


Although one would expect the gradients direction to use the same subject coordinate system as the one of `ACQ_grad_matrix` (i.e. RDH), `PVM_DwBMatPat` and `VisuAcqDiffusionBMatrix` need an extra transform (flipping the F-H axis).

In [11]:
flip_f_h_axis = numpy.asarray([
    [1, 0,  0],
    [0, 1,  0],
    [0, 0, -1]])

print("Minimal colinearity between transformed PVM_DwDir and PVM_DwBMatPat")
report_colinearity(
    PVM_DwDir, PVM_DwBMatPat, 
    lambda x,y: ACQ_grad_matrix[x][y][0].dot(flip_f_h_axis), b_value_threshold)

Minimal colinearity between transformed PVM_DwDir and PVM_DwBMatPat
  rat/6: 0.995793158996 (5.25736400799 degrees)
  rat/8: 0.995793159801 (5.25736350469 degrees)
  rat/9: 0.995793159616 (5.25736362048 degrees)
  rat/10: 0.995793159616 (5.25736362048 degrees)
  rat/11: 0.995793159616 (5.25736362048 degrees)
  rat/12: 0.995793159616 (5.25736362048 degrees)
  rat/13: 0.995793159102 (5.25736394139 degrees)
  saimiri/7: 0.998381072353 (3.26069335517 degrees)


### Magnet coordinate system

The directions defined in `PVM_DwDir` can be converted to the magnet coordinate system described above by combining the transforms stored in `ACQ_grad_matrix` and `ACQ_patient_pos`. 

In [12]:
PVM_DwBMatMag = {}
for name, series in meta_data.items():
    for series_number, series_meta_data in series.items():
        PVM_DwBMatMag.setdefault(name, {})[series_number] = get_array(
            series_meta_data["method"].get_field("PVM_DwBMatMag"))

print("Minimal colinearity between transformed PVM_DwDir and PVM_DwBMatMag")
report_colinearity(
    PVM_DwDir, PVM_DwBMatMag, 
    lambda x,y: ACQ_grad_matrix[x][y][0].dot(ACQ_patient_pos[x][y]), 
    b_value_threshold)

Minimal colinearity between transformed PVM_DwDir and PVM_DwBMatMag
  rat/6: 0.995793158996 (5.25736400799 degrees)
  rat/8: 0.995793159801 (5.25736350469 degrees)
  rat/9: 0.995793159616 (5.25736362048 degrees)
  rat/10: 0.995793159616 (5.25736362048 degrees)
  rat/11: 0.995793159616 (5.25736362048 degrees)
  rat/12: 0.995793159616 (5.25736362048 degrees)
  rat/13: 0.995793159102 (5.25736394139 degrees)
  saimiri/7: 0.998381072353 (3.26069335517 degrees)


### Image coordinate system

The directions defined in `PVM_DwDir` can be converted to the image coordinate system described above by combining `ACQ_grad_matrix`, the F-H axis flip and `VisuCoreOrientation`. 

In [13]:
PVM_DwBMatImag = {}
for name, series in meta_data.items():
    for series_number, series_meta_data in series.items():
        PVM_DwBMatImag.setdefault(name, {})[series_number] = get_array(
            series_meta_data["method"].get_field("PVM_DwBMatImag"))

print("Minimal colinearity between transformed PVM_DwDir and PVM_DwBMatImag")
report_colinearity(
    PVM_DwDir, PVM_DwBMatImag, 
    # WARNING: VisuCoreOrientation has a column-vector formalism
    lambda x,y: ACQ_grad_matrix[x][y][0].dot(flip_f_h_axis).dot(VisuCoreOrientation[x][y][0].T),
    b_value_threshold)

Minimal colinearity between transformed PVM_DwDir and PVM_DwBMatImag
  rat/6: 0.99579315961 (5.25736362369 degrees)
  rat/8: 0.99579315961 (5.25736362369 degrees)
  rat/9: 0.995793159616 (5.25736362048 degrees)
  rat/10: 0.995793159616 (5.25736362048 degrees)
  rat/11: 0.995793159616 (5.25736362048 degrees)
  rat/12: 0.995793159616 (5.25736362048 degrees)
  rat/13: 0.995793159538 (5.25736366913 degrees)
  saimiri/7: 0.998381072353 (3.26069335517 degrees)


## Interoperability with other standards and diffusion softwares

### DICOM

In [DICOM](http://dicom.nema.org/medical/dicom/current/output/chtml/part03/sect_C.8.13.5.9.html), the direction of the diffusion gradient and the b-matrix are explicitely specified in the [LDH](http://dicom.nema.org/medical/dicom/current/output/chtml/part03/sect_C.7.6.2.html#sect_C.7.6.2.1.1) subject coordinate system. 

In [14]:
rdh_to_ldh = numpy.asarray([
    [-1, 0, 0],
    [ 0, 1, 0],
    [ 0, 0, 1]])
dicom_meta_data = {}
for name, series in PVM_DwGradVec.items():
    for series_number, dw_grad_vecs in series.items():
        orientations = [
            x.dot(ACQ_grad_matrix[name][series_number][0]).dot(rdh_to_ldh)
            for x in dw_grad_vecs]
        b_values = PVM_DwEffBval[name][series_number]
        dicom_meta_data.setdefault(name, {})[series_number] = {
            "MRDiffusionSequence": [{
                "DiffusionBValue": [b_value],
                "DiffusionGradientDirectionSequence": [{
                    "DiffusionGradientOrientation": orientation.tolist()
                }]
            } for orientation, b_value in zip(orientations, b_values)]
        }

### FSL: `bvecs` and `bvals` files

FSL uses a [radiological voxel convention](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FDT/FAQ#What_conventions_do_the_bvecs_use.3F). This rather convoluted expression means that the directions are in the image coordinate system if the image-to-subject transformation has a negative determinant. If this determinant is positive, then an additional flip on the x axis must be applied to match the "old Analyze convention". As an extra indication that the directions are stored in the image coordinate system and not in the subject coordinate system, both the [Eddy FAQ](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/eddy/Faq#Will_eddy_rotate_my_bvecs_for_me.3F) and a [practical in the FSL course](http://fsl.fmrib.ox.ac.uk/fslcourse/lectures/practicals/fdt1/index.html) mention rotation of the direction when performing motion correction, i.e. being image dependent. MRtrix also [applies this process](https://github.com/MRtrix3/mrtrix3/blob/master/src/dwi/gradient.cpp#L127-L133) the other way around to convert from the `bvecs` file to their gradient scheme.

We first compute the NIfTI orientation matrix from `VisuCoreOrientation`.

In [15]:
ldh_to_rvh = numpy.asarray([
    [-1,  0, 0],
    [ 0, -1, 0],
    [ 0,  0, 1]])

nifti_orientation_matrix = {}
for name, series in VisuCoreOrientation.items():
    for series_number, orientation in series.items():
        nifti_orientation_matrix.setdefault(name, {})[series_number] = numpy.dot(
            ldh_to_rvh, orientation[0])

The `bvecs` and `bvals` are then given by:

In [16]:
bvecs = {}
bvals = {}
for name, series in dicom_meta_data.items():
    for series_number, series_dicom_meta_data in series.items():
        bvecs.setdefault(name, {})[series_number] = []
        bvals.setdefault(name, {})[series_number] = []
        series_nifti_orientation_matrix = nifti_orientation_matrix[name][series_number]
        for entry in series_dicom_meta_data["MRDiffusionSequence"]:
            # In LDH
            orientation = entry["DiffusionGradientDirectionSequence"][0]["DiffusionGradientOrientation"]
            # To RVH
            orientation = numpy.dot(ldh_to_rvh, orientation)
            # To image
            orientation = numpy.dot(series_nifti_orientation_matrix, orientation)
            # To Analyze convention, if required
            if numpy.linalg.det(series_nifti_orientation_matrix) > 0:
                orientation[0] *= -1
    
            bvecs[name][series_number].append(orientation)
            bvals[name][series_number].append(entry["DiffusionBValue"][0])
        bvecs[name][series_number] = numpy.transpose(bvecs[name][series_number])

for name, series in bvecs.items():
    for series_number, series_bvecs in series.items():
        series_bvals = bvals[name][series_number]
        nifti_directory = glob.glob(
            os.path.join(name, "nifti", "*", "*", "{}*".format(
                2**16 * series_number + 1)))[0]
        with open(os.path.join(nifti_directory, "bvecs"), "w") as fd:
            numpy.savetxt(fd, numpy.transpose(series_bvecs))
        with open(os.path.join(nifti_directory, "bvals"), "w") as fd:
            numpy.savetxt(fd, numpy.reshape(series_bvals, (1, -1)))

In order to check the correctness of the `bvecs` and `bvals`, we use FSL's own `dtifit`:

```shell
bet dwi.nii.gz brain -m
dtifit -k dwi.nii.gz -r bvecs -b bvals -o fsl -m brain_mask.nii.gz
```

In [17]:
for name, series in bvecs.items():
    for series_number, series_bvecs in series.items():
        nifti_directory = glob.glob(
            os.path.join(name, "nifti", "*", "*", "{}*".format(
                2**16 * series_number + 1)))[0]
        dwi = os.path.join(nifti_directory, "1.nii.gz")
        brain_mask = os.path.join(nifti_directory, "brain_mask.nii.gz")
        
        if (
                not os.path.isfile(brain_mask) 
                or os.stat(brain_mask).st_mtime < os.stat(dwi).st_mtime):
            subprocess.check_call(
                ["bet", "1.nii.gz", "brain", "-m"], cwd=nifti_directory)
        
        fa = os.path.join(nifti_directory, "fsl_FA.nii.gz")
        if (
                not os.path.isfile(fa) 
                or os.stat(fa).st_mtime < os.stat(dwi).st_mtime):
            subprocess.check_call(
                [
                    "dtifit", 
                    "-k", "1.nii.gz", "-m", "brain_mask.nii.gz",
                    "-r", "bvecs", "-b", "bvals", 
                    "-o", "fsl"],
                cwd=nifti_directory)

The data can then be visualized with `fsleyes`, displaying the first eigenvector on top of the fractional anisotropy map:
```
fsleyes --layout grid fsl_FA.nii.gz fsl_V1.nii.gz -ot linevector
```

**WARNING** the voxel size of the rat data set is too anisotropic (0.209 x 0.180 x 1) for the tensor estimation to have a high angular resolution. See if we can get data with a less anisotropic resolution.

In [18]:
for name, series in bvecs.items():
    for series_number, series_bvecs in series.items():
        nifti_directory = glob.glob(
            os.path.join(name, "nifti", "*", "*", "{}*".format(
                2**16 * series_number + 1)))[0]
        print "fsleyes --layout grid {}/fsl_{{FA,V1}}.nii.gz -ot linevector".format(nifti_directory)

fsleyes --layout grid rat/nifti/R17-06/2_plateforme^14112017/393217_DTI_EPI_seg_30dir/fsl_{FA,V1}.nii.gz -ot linevector
fsleyes --layout grid rat/nifti/R17-06/2_plateforme^14112017/524289_DTI_EPI_seg_30dir/fsl_{FA,V1}.nii.gz -ot linevector
fsleyes --layout grid rat/nifti/R17-06/2_plateforme^14112017/589825_DTI_EPI_seg_30dir/fsl_{FA,V1}.nii.gz -ot linevector
fsleyes --layout grid rat/nifti/R17-06/2_plateforme^14112017/655361_DTI_EPI_seg_30dir/fsl_{FA,V1}.nii.gz -ot linevector
fsleyes --layout grid rat/nifti/R17-06/2_plateforme^14112017/720897_DTI_EPI_seg_30dir/fsl_{FA,V1}.nii.gz -ot linevector
fsleyes --layout grid rat/nifti/R17-06/2_plateforme^14112017/786433_DTI_EPI_seg_30dir/fsl_{FA,V1}.nii.gz -ot linevector
fsleyes --layout grid rat/nifti/R17-06/2_plateforme^14112017/851969_DTI_EPI_seg_30dir/fsl_{FA,V1}.nii.gz -ot linevector
fsleyes --layout grid saimiri/nifti/20170117-Saimiri-07011/1_20170117-Saimiri-07011/458753_DTI_60dirs_new/fsl_{FA,V1}.nii.gz -ot linevector


### MRtrix

The [MRtrix gradient file](http://mrtrix.readthedocs.io/en/latest/concepts/dw_scheme.html#mrtrix-format) has conflicting informations about the coordinate system it uses: it mentions both "scanner coordinates" and "as DICOM". However, when [loading NIfTI files](https://github.com/MRtrix3/mrtrix3/blob/master/core/file/nifti2_utils.cpp#L158-L203), only the `sform` or `qform` is used to establish the [MRtrix image transform](https://github.com/MRtrix3/mrtrix3/blob/master/core/file/nifti2_utils.cpp#L158-L203). Diffusion data from NIfTI is [loaded](https://github.com/MRtrix3/mrtrix3/blob/master/src/dwi/gradient.cpp#L98-L152) by transform the `bvec` data to the RAS subject coordinate system.

In [19]:
orientation_mrtrix = [
    numpy.dot(
        ldh_to_rvh, 
        x["DiffusionGradientDirectionSequence"][0]["DiffusionGradientOrientation"]) 
    for x in meta_data["MRDiffusionSequence"]]
mrtrix_scheme = numpy.hstack((
    orientation_mrtrix, [x["DiffusionBValue"] for x in meta_data["MRDiffusionSequence"]]))
with BytesIO() as fd:
    numpy.savetxt(fd, mrtrix_scheme)
    print(fd.getvalue())

KeyError: 'MRDiffusionSequence'

Correctness can be checked with `dwi2tensor`:
```shell
dwi2tensor -grad mrtrix.scheme -mask brain_mask.nii.gz dwi.nii.gz mrtrix_DT.nii.gz
tensor2metric -fa mrtrix_FA.nii.gz mrtrix_DT.nii.gz
tensor2metric -vector mrtrix_V1.nii.gz -num 1 mrtrix_DT.nii.gz
mrview mrtrix_FA.nii.gz -fixel.load mrtrix_V1.nii.gz -mode 2
```

### Camino